In [1]:
import tensorflow as tf

In [2]:
data_path = "/Users/suparnac/dev_envs/CV_Projects/satellite_image_classifier/EuroSAT"

In [3]:
# preprocessing the dataset
# split the data into train and test/validation

train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_path,
    validation_split=0.2, # 80-20 dataset split
    subset = "training", # specifies this dataset will be training subset and not the validation
    seed = 123, #sets random seed for shuffling and splitting, ensuring reproducibility
    image_size = (128, 128), #resize images intop 128x128 for easy processing
    batch_size = 32 #groups images into batches of 32 for training
)

Found 27002 files belonging to 10 classes.
Using 21602 files for training.


2025-07-08 03:39:12.172726: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-07-08 03:39:12.172748: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-07-08 03:39:12.172750: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-07-08 03:39:12.172765: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-07-08 03:39:12.172773: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    data_path,
    validation_split =0.2,
    subset ="validation", # previously it considered the dataset for the training, here it is assigning the data for validation
    seed = 123,
    image_size = (128, 128),
    batch_size = 32
)

Found 27002 files belonging to 10 classes.
Using 5400 files for validation.


In [5]:
''' normalizing the images to range [0,1]
need to normalize:  
- stability and speed of training
- consistent data range
- improved model performance
- compatibility with activation functions 
'''

normalization_layer = tf.keras.layers.Rescaling(1./255) # rescaling layer from the keras API
# map() applies function to each element (batch) in the dataset
train_dataset = train_dataset.map(lambda x,y: (normalization_layer(x), y)) # x - here images , y - here labels 
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))

In [6]:
#building the model using transfer learning
base_model = tf.keras.applications.MobileNetV2(input_shape = (128,128,3), include_top= False, weights='imagenet')

''' input shape = 128,128, 3 --- 3 here is the 3 color channels (RGB)
include_top = False --- determines whether to include the fully connected layers (the "top") layers of the original model (here MobileNetV2)
weights = 'imagenet', (pre-trained weights from ImageNet) Loads pre-trained weights from training on the ImageNet dataset.
ImageNet is a large visual database. The model initializes with weights learned from the large-scale ImageNet dataset.
'''

base_model.trainable = False #the pre-trained MobileNetV2 model will not be updated during training. Its weights will remain fixed (frozen), and only the additional layers added afterward will be trained.

In [7]:
model = tf.keras.Sequential(
    [base_model, # above trined base_model using MobileNetV2
     tf.keras.layers.GlobalAveragePooling2D(), # Reduces each feature map to a single value by averaging, converting the output into a 1D vector.
     tf.keras.layers.Dense(128, activation = 'relu'), #Adds a fully connected layer with 128 neurons and ReLU activation, introducing non-linearity.
     tf.keras.layers.Dense(10, activation = 'softmax') #Adds an output layer with 10 neurons (for 10 classes) using softmax activation to produce probability scores for each class.
    ])

In [11]:
model.compile(optimizer='adam', #Adam(Adaptive Moment Estimation)--optimizer algorithm adjusts a neural network's weights during training to minimize the errors(loss),  
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

In [12]:
#train the model
training_history = model.fit(train_dataset, epochs=5)


Epoch 1/5


2025-07-08 03:40:23.642133: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


676/676 ━━━━━━━━━━━━━━━━━━━━ 28s 36ms/step - accuracy: 0.7963 - loss: 0.6104
Epoch 2/5
676/676 ━━━━━━━━━━━━━━━━━━━━ 23s 35ms/step - accuracy: 0.8942 - loss: 0.3115
Epoch 3/5
676/676 ━━━━━━━━━━━━━━━━━━━━ 24s 35ms/step - accuracy: 0.9056 - loss: 0.2731
Epoch 4/5
676/676 ━━━━━━━━━━━━━━━━━━━━ 24s 35ms/step - accuracy: 0.9048 - loss: 0.2753
Epoch 5/5
676/676 ━━━━━━━━━━━━━━━━━━━━ 24s 35ms/step - accuracy: 0.9002 - loss: 0.3129


In [13]:
#save the model

model.save("satellite_classifier_model.h5")
